# Harmonic Pattern Recognition

This notebook demonstrates numta's harmonic pattern detection capabilities, which identify price structures based on Fibonacci ratios.

In [ ]:
import numpy as np
import pandas as pd
import numta
from numta import (
    # Fibonacci utilities
    fibonacci_retracement, fibonacci_extension,
    # Harmonic pattern detection
    detect_gartley, detect_butterfly, detect_bat, detect_crab,
    detect_harmonic_patterns,
    HarmonicPattern
)

## Fibonacci Ratios in Trading

Harmonic patterns are based on Fibonacci ratios. The key ratios are:

| Ratio | Description |
|-------|-------------|
| 0.236 | 23.6% retracement |
| 0.382 | 38.2% retracement |
| 0.500 | 50% retracement |
| 0.618 | 61.8% retracement (Golden Ratio) |
| 0.786 | 78.6% retracement |
| 0.886 | 88.6% retracement |
| 1.272 | 127.2% extension |
| 1.618 | 161.8% extension |

In [ ]:
# Calculate Fibonacci retracement levels
high_price = 150.0
low_price = 100.0

levels = fibonacci_retracement(high_price, low_price)
print("Fibonacci Retracement Levels:")
for level, price in levels.items():
    print(f"  {level*100:.1f}%: {price:.2f}")

In [ ]:
# Calculate Fibonacci extension levels
extensions = fibonacci_extension(high_price, low_price)
print("Fibonacci Extension Levels:")
for level, price in extensions.items():
    print(f"  {level*100:.1f}%: {price:.2f}")

## Creating Sample Data

In [ ]:
np.random.seed(42)
n = 500

# Generate price data with multiple waves
t = np.linspace(0, 8*np.pi, n)
trend = np.sin(t) * 15 + np.sin(t/3) * 10
noise = np.cumsum(np.random.randn(n) * 0.3)
close = 100 + trend + noise

# Create OHLC
df = pd.DataFrame({
    'open': close + np.random.randn(n) * 0.3,
    'high': close + np.abs(np.random.randn(n)) * 1.0,
    'low': close - np.abs(np.random.randn(n)) * 1.0,
    'close': close
})

highs = df['high'].values
lows = df['low'].values

print(f"Sample data: {n} bars")

## Gartley Pattern

The Gartley pattern, discovered by H.M. Gartley in 1935, is a 5-point pattern with specific Fibonacci ratios:

- **AB retracement of XA**: 61.8%
- **BC retracement of AB**: 38.2% - 88.6%
- **CD extension of BC**: 127.2% - 161.8%
- **D retracement of XA**: 78.6%

In [ ]:
# Detect Gartley patterns
gartley_patterns = detect_gartley(highs, lows, order=5)

print(f"Gartley patterns found: {len(gartley_patterns)}")

for pattern in gartley_patterns[:3]:
    print(f"\nGartley Pattern ({pattern.direction})")
    print(f"  Points: X={pattern.x_index}, A={pattern.a_index}, B={pattern.b_index}, C={pattern.c_index}, D={pattern.d_index}")
    print(f"  Confidence: {pattern.confidence:.2f}")
    print(f"  PRZ (Potential Reversal Zone): {pattern.prz:.2f}")

## Butterfly Pattern

The Butterfly pattern extends beyond the initial XA leg:

- **AB retracement of XA**: 78.6%
- **BC retracement of AB**: 38.2% - 88.6%
- **CD extension of BC**: 161.8% - 224%
- **D extension of XA**: 127% - 161.8%

In [ ]:
# Detect Butterfly patterns
butterfly_patterns = detect_butterfly(highs, lows, order=5)

print(f"Butterfly patterns found: {len(butterfly_patterns)}")

for pattern in butterfly_patterns[:3]:
    print(f"\nButterfly Pattern ({pattern.direction})")
    print(f"  Confidence: {pattern.confidence:.2f}")
    print(f"  PRZ: {pattern.prz:.2f}")

## Bat Pattern

The Bat pattern has a deep retracement at point D:

- **AB retracement of XA**: 38.2% - 50%
- **BC retracement of AB**: 38.2% - 88.6%
- **CD extension of BC**: 161.8% - 261.8%
- **D retracement of XA**: 88.6%

In [ ]:
# Detect Bat patterns
bat_patterns = detect_bat(highs, lows, order=5)

print(f"Bat patterns found: {len(bat_patterns)}")

for pattern in bat_patterns[:3]:
    print(f"\nBat Pattern ({pattern.direction})")
    print(f"  Confidence: {pattern.confidence:.2f}")
    print(f"  PRZ: {pattern.prz:.2f}")

## Crab Pattern

The Crab pattern has an extreme extension at point D:

- **AB retracement of XA**: 38.2% - 61.8%
- **BC retracement of AB**: 38.2% - 88.6%
- **CD extension of BC**: 224% - 361.8%
- **D extension of XA**: 161.8%

In [ ]:
# Detect Crab patterns
crab_patterns = detect_crab(highs, lows, order=5)

print(f"Crab patterns found: {len(crab_patterns)}")

for pattern in crab_patterns[:3]:
    print(f"\nCrab Pattern ({pattern.direction})")
    print(f"  Confidence: {pattern.confidence:.2f}")
    print(f"  PRZ: {pattern.prz:.2f}")

## Detecting All Harmonic Patterns

In [ ]:
# Detect all harmonic patterns at once
all_harmonics = detect_harmonic_patterns(highs, lows, order=5)

print(f"Total harmonic patterns found: {len(all_harmonics)}")

# Group by pattern type
from collections import Counter
pattern_counts = Counter(p.pattern_type for p in all_harmonics)
print("\nPatterns by type:")
for ptype, count in pattern_counts.items():
    print(f"  {ptype}: {count}")

## Potential Reversal Zones (PRZ)

The PRZ is the area where the pattern completes and a reversal is expected. It's calculated based on the Fibonacci levels of point D.

In [ ]:
# Sort by confidence and show PRZ levels
all_harmonics.sort(key=lambda x: x.confidence, reverse=True)

print("Top harmonic patterns by confidence:")
for pattern in all_harmonics[:5]:
    direction = 'Bullish' if pattern.direction == 'bullish' else 'Bearish'
    print(f"\n{pattern.pattern_type} ({direction})")
    print(f"  Confidence: {pattern.confidence:.3f}")
    print(f"  PRZ (entry zone): {pattern.prz:.2f}")
    print(f"  Pattern indices: X={pattern.x_index}, A={pattern.a_index}, B={pattern.b_index}, C={pattern.c_index}, D={pattern.d_index}")

## Using the Pandas Accessor

In [ ]:
# Find all harmonic patterns
harmonics = df.ta.find_harmonic_patterns()
print(f"All harmonics: {len(harmonics)}")

# Find specific patterns
gartley = df.ta.find_harmonic_patterns(patterns=['gartley'])
print(f"Gartley patterns: {len(gartley)}")

bat_crab = df.ta.find_harmonic_patterns(patterns=['bat', 'crab'])
print(f"Bat + Crab patterns: {len(bat_crab)}")

## Trading with Harmonic Patterns

Key considerations when trading harmonic patterns:

1. **PRZ as Entry Zone**: Enter trades when price reaches the PRZ
2. **Confirmation**: Wait for price action confirmation (candlestick patterns)
3. **Stop Loss**: Place stops beyond the PRZ
4. **Take Profit**: Use Fibonacci extensions for targets
5. **Confidence Score**: Higher confidence patterns are more reliable

In [ ]:
# Example: Filter high-confidence patterns
high_confidence = [p for p in all_harmonics if p.confidence > 0.7]
print(f"High confidence patterns (>0.7): {len(high_confidence)}")

# Separate by direction
bullish = [p for p in high_confidence if p.direction == 'bullish']
bearish = [p for p in high_confidence if p.direction == 'bearish']
print(f"  Bullish: {len(bullish)}")
print(f"  Bearish: {len(bearish)}")

## Next Steps

- See `07_visualization.ipynb` for visualizing harmonic patterns with lwcharts
- See `06_streaming_indicators.ipynb` for real-time pattern detection